In [ ]:
import cadquery as cq
from cadquery import Vector, Location
import math

# Set parameters
n = 2 # Number of pins
P = 3.96 # mm pin pitch
C1 = 6.8 # mm Height of bottom plate
H1 = 9.4 # mm Height of latching plate
O1 = 2 # mm Vertical offset pin-center to bottom plate border
D1 = 3.2 # mm Total depth of bottom plate
Pw = 1.14 # mm Width & height of square pin
D2 = 1.3 # mm (Measured) depth of latching plate (narrowest section)
Wx1 = 1.0 # mm (Measured) Width of the topmost part of the latching plate
Hx1 = 0.5 # mm (Measured) Height of the latching surface facing away from the pins
alpha = 90 # ° Angle of the latching surface. Shown by the datasheet as ~45°but real parts have 90°
beta = 15 # ° (Estimated) Latching plate tapering angle
WL1 = 1.27 # mm (Measured) With of the section of the bottom plate where there is no latching plate per side)
Dlatch = 1.75 # mm (Measured) Total depth of latching plate
delta = 70 # ° (Estimated) Taper angle of the pin top & bottoms
FilletRadiusBaseplate = 0.4 # mm

# Compute derived variables
A1 = ( n - 1 ) * P # Total width of all pins (center of last pin distance to center of first pin)
B1 = A1 + ( 1.95 * 2 ) # Width of bottom plate
LLatch = B1 - ( WL1 * 2 ) # Length of the latching plate
LPinTop = 7.7 + D1 # Length of the pin measured from the bottom plane to the top of the pin (away from PCB)
LPinBottom = 14.6 - LPinTop # Length of the pin in bottom direction, measured from the bottom plane
Pin1XOffset = ( ( n - 1 ) * P ) / 2 # First pin X offset measured from the X axis


In [ ]:
import cadquery as cq

base_box = cq.Workplane("XY").box(B1, C1, D1).translate((0, 0, D1 / 2)) # Bottom of plate => Z=0
base_box = base_box.edges("|Z").fillet(FilletRadiusBaseplate)
base_box

In [ ]:
# Compute asymmetric chamfer with constant angle triangle lengths
# Note: Top-facing angle of the chamfering triangle is [beta]
opposing_side = (Dlatch - Wx1) / 2 # Compute so that remaining part of latch is Wx1 wide
adjacent_side = opposing_side * (1/math.tan(math.radians(beta)))

# Start just outside the bottom plate
xstart = C1/2
xend = xstart + Dlatch
h1MinusChamfer = H1 - adjacent_side
latchPlate = (cq.Workplane("YZ")
    .sketch()
    # From bottom to start of chamfer
    .segment((xstart, 0), (xstart, h1MinusChamfer))
    # Chamfer edge
    .segment((xstart + opposing_side, H1))
    # Top side of latch (straight line) (up to begin of other side chamfer)
    .segment((xend - opposing_side, H1))
    # Chamfer edge
    .segment((xend, h1MinusChamfer))
    # move down to begin of latching surface
    .segment((xend, h1MinusChamfer - Hx1))
    # Move inwards (-X) for latching surface. NOTE: we assume 90° angle here
    # NOTE: D2 is the depth of the latching plate below the latching surface
    .segment((xstart + D2, h1MinusChamfer - Hx1))
    # From latching surface inside to bottom
    .segment((xstart + D2, 0))
    .close()
    .assemble(tag="face")
    .finalize()
    .extrude(LLatch/2.0, both=True)
)
plastic = latchPlate.union(base_box)
plastic

In [ ]:
# Create pins
pin_sketch = (
    cq.Workplane("XY")
    .sketch()
    .push([cq.Location((Pin1XOffset, (C1/2)-O1, 0))])
    .rect(Pw, Pw)
    .finalize()
)
pin = pin_sketch.extrude(LPinTop).union(pin_sketch.extrude(-LPinBottom))
# Chamfer top & bottom
pin = pin.faces(">Z").chamfer(0.2)
pin = pin.faces("<Z").chamfer(0.2)
pin.union(plastic)

In [ ]:
obj.edges(">Z").edges("|X").chamfer(0.5)

In [ ]:
import cadquery as cq

import cadquery as cq

xstart = 1.0
height = 4.0
width = 1.0
obj = cq.Workplane("YZ")\
    .sketch()\
    .line(0, height)\
    .line(width, 0)\
    .line(0, -height)\
    .close()\
    .assemble(tag="face")\
    .finalize()\
    .extrude(0.1)
obj